# CRS handling

RasterIndex composes with [xproj](https://xproj.readthedocs.io) to provide CRS handling.

In [ ]:
%xmode minimal

import xarray as xr

import rasterix

xr.set_options(display_expand_indexes=True)

In [ ]:
source = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"

To enable CRS-handling, start by assigning a CRS using the `.proj.assign_crs` function from xproj.

In [ ]:
da = xr.open_dataarray(source, engine="rasterio")
da = da.proj.assign_crs(spatial_ref=da.spatial_ref.attrs["crs_wkt"])
da = da.pipe(rasterix.assign_index)
da

Note how the `x` variable has appropriate attributes!

In [ ]:
da.x.attrs

## Alignment

For demo purposes we'll create a second copy with a different CRS.

In [ ]:
diffcrs = da.copy(deep=True).drop_vars("spatial_ref").drop_indexes(["x", "y"])
diffcrs = diffcrs.proj.assign_crs(spatial_ref="epsg:4326").pipe(rasterix.assign_index)
diffcrs

Again note that the attributes on `x` have changed appropriately. This is enabled by RasterIndex's integration with `xproj`

In [ ]:
diffcrs.x.attrs

Attempting to add the two raises an error (as it should)!

In [ ]:
diffcrs + da